In [ ]:
import geojson
import geopandas as gpd
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint
from geopandas.tools import sjoin
import matplotlib.cm as cm
import matplotlib.pyplot as plt # plotting
import seaborn as sns; sns.set()
from IPython.display import Image
from branca.colormap import  linear
import json
import branca.colormap as cm
import folium
import numpy as np 
import os 
import pandas as pd 
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import tensorflow_probability as tfp
import gc
import time

In [ ]:
df = pd.read_csv('/kaggle/input/sample-obu/sample_obu_new.csv', header=None)

In [ ]:
df.head()

In [ ]:
geometry = [Point(xy) for xy in zip(df[2],df[3])]

In [ ]:
df_geo = gpd.GeoDataFrame(df, geometry = geometry)
df_geo = df_geo.set_crs(epsg=4326)
df_geo.head()

In [ ]:
from folium.plugins import MarkerCluster

m = folium.Map(location=[50.85045, 4.34878],  zoom_start=11)

marker_cluster = MarkerCluster().add_to(m)

for coord in range(len(df_geo.iloc[2000:3000])):
    folium.Marker( location=[df[3][coord], df[2][coord] ]).add_to(marker_cluster)
m

In [ ]:
df_belgium = gpd.read_file('/kaggle/input/sample-obu/Belgium.json')['geometry']
df_bruxelles = gpd.read_file('/kaggle/input/belgium-obu/bruxelles.json')
df_anderlecht = gpd.read_file('/kaggle/input/sample-obu/Anderlecht.json')
df_ville = gpd.read_file('/kaggle/input/sample-obu/Bruxelles-ville.json')

In [ ]:
df_belgium = gpd.GeoDataFrame(pd.DataFrame(df_belgium, columns=['geometry']).iloc[0]).T
df_belgium = df_belgium.set_crs(epsg=4326)

In [ ]:
m = folium.Map([50.85045, 4.34878], zoom_start=7, tiles='cartodbpositron')
folium.GeoJson(df_belgium).add_to(m)
folium.GeoJson(df_bruxelles).add_to(m)
folium.GeoJson(df_anderlecht).add_to(m)
folium.GeoJson(df_ville).add_to(m)
folium.LayerControl().add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
def create_hexagon(l, x, y):
    """
    Create a hexagon centered on (x, y)
    :param l: length of the hexagon's edge
    :param x: x-coordinate of the hexagon's center
    :param y: y-coordinate of the hexagon's center
    :return: The polygon containing the hexagon's coordinates
    """
    c = [[x + math.cos(math.radians(angle)) * l, y + math.sin(math.radians(angle)) * l] for angle in range(0, 360, 60)]
    return Polygon(c)


def create_hexgrid(bbox, side):
    """
    returns an array of Points describing hexagons centers that are inside the given bounding_box
    :param bbox: The containing bounding box. The bbox coordinate should be in Webmercator.
    :param side: The size of the hexagons'
    :return: The hexagon grid
    """
    grid = []

    v_step = math.sqrt(3) * side
    h_step = 1.5 * side

    x_min = min(bbox[0], bbox[2])
    x_max = max(bbox[0], bbox[2])
    y_min = min(bbox[1], bbox[3])
    y_max = max(bbox[1], bbox[3])

    h_skip = math.ceil(x_min / h_step) - 1
    h_start = h_skip * h_step

    v_skip = math.ceil(y_min / v_step) - 1
    v_start = v_skip * v_step

    h_end = x_max + h_step
    v_end = y_max + v_step

    if v_start - (v_step / 2.0) < y_min:
        v_start_array = [v_start + (v_step / 2.0), v_start]
    else:
        v_start_array = [v_start - (v_step / 2.0), v_start]

    v_start_idx = int(abs(h_skip) % 2)

    c_x = h_start
    c_y = v_start_array[v_start_idx]
    v_start_idx = (v_start_idx + 1) % 2
    while c_x < h_end:
        while c_y < v_end:
            grid.append((c_x, c_y))
            c_y += v_step
        c_x += h_step
        c_y = v_start_array[v_start_idx]
        v_start_idx = (v_start_idx + 1) % 2

    return grid

In [ ]:
import json
import math 

with open('/kaggle/input/belgium-obu/bruxelles.json') as f:
    data = json.load(f)
    
with open('/kaggle/input/sample-obu/Anderlecht.json') as f:
    data_ander = json.load(f)
    
with open('/kaggle/input/sample-obu/Bruxelles-ville.json') as f:
    data_ville = json.load(f)

In [ ]:
import shapely 

def create_squargrid(bbox, numb_cells):
    
    # total area for the grid
    xmin, ymin, xmax, ymax= bbox #df_bruxelles.total_bounds
    # how many cells across and down
    n_cells = numb_cells
    cell_size = (xmax-xmin)/n_cells
    # create the cells in a loop
    grid_cells = []
    for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
        for y0 in np.arange(ymin, ymax+cell_size, cell_size):
            # bounds
            x1 = x0-cell_size
            y1 = y0+cell_size
            grid_cells.append( shapely.geometry.box(x0, y0, x1, y1) )
    grid = gpd.GeoDataFrame(grid_cells, columns=['geometry']).set_crs(epsg=4326)
    return grid
    


In [ ]:
edge = math.sqrt(0.05**2/(3/2 * math.sqrt(3)))
hex_centers = create_hexgrid(df_belgium.bounds.values[0], edge)
bel_hex_lists = [ ]
for center in hex_centers:
    hexagon = create_hexagon(edge, center[0], center[1])
    bel_hex_lists.append(hexagon)

In [ ]:
str(bel_hex_lists[0])

In [ ]:

# df_bruxelles = df_bruxelles.to_crs(epsg=3857)

edge = math.sqrt(0.005**2/(3/2 * math.sqrt(3)))
hex_centers = create_hexgrid(df_bruxelles.bounds.values[0], edge)
bxl_hex_lists = [ ]
for center in hex_centers:
    hexagon = create_hexagon(edge, center[0], center[1])
    bxl_hex_lists.append(hexagon)

# df_bruxelles = df_bruxelles.to_crs(epsg=4326)

In [ ]:
# df_anderlecht = df_anderlecht.to_crs(epsg=3857)

edge = math.sqrt(0.001**2/(3/2 * math.sqrt(3)))
hex_centers = create_hexgrid(df_anderlecht.bounds.values[0], edge)
ander_hex_lists = [ ]
for center in hex_centers:
    hexagon = create_hexagon(edge, center[0], center[1])
    ander_hex_lists.append(hexagon)

# df_anderlecht = df_anderlecht.to_crs(epsg=4326)

In [ ]:
edge = math.sqrt(0.001**2/(3/2 * math.sqrt(3)))
hex_centers = create_hexgrid(df_ville.bounds.values[0], edge)
ville_hex_lists = [ ]
for center in hex_centers:
    hexagon = create_hexagon(edge, center[0], center[1])
    ville_hex_lists.append(hexagon)


In [ ]:
bel_df_hexagons = gpd.GeoDataFrame(bel_hex_lists, columns=['geometry'])
# bel_df_hexagons = bel_df_hexagons.set_crs(epsg=3857)
bel_df_hexagons = bel_df_hexagons.set_crs(epsg=4326)
bel_df_hexagons.plot()

In [ ]:
bxl_df_hexagons = gpd.GeoDataFrame(bxl_hex_lists, columns=['geometry'])
# bxl_df_hexagons = bxl_df_hexagons.set_crs(epsg=3857)
bxl_df_hexagons = bxl_df_hexagons.set_crs(epsg=4326)
bxl_df_hexagons.plot()

In [ ]:
ander_df_hexagons = gpd.GeoDataFrame(ander_hex_lists, columns=['geometry'])
# ander_df_hexagons = ander_df_hexagons.set_crs(epsg=3857)
ander_df_hexagons = ander_df_hexagons.set_crs(epsg=4326)
ander_df_hexagons.plot()

In [ ]:
ville_df_hexagons = gpd.GeoDataFrame(ville_hex_lists, columns=['geometry'])
# ander_df_hexagons = ander_df_hexagons.set_crs(epsg=3857)
ville_df_hexagons = ville_df_hexagons.set_crs(epsg=4326)
ville_df_hexagons.plot()

In [ ]:
cell_bel = create_squargrid(df_belgium.total_bounds, 100)
cell_bxl = create_squargrid(df_bruxelles.total_bounds, 75)

In [ ]:
bel_final_hxgn = [ ]
for i in bel_df_hexagons['geometry']:
    if i.intersects(df_belgium['geometry'][0])==True:
        bel_final_hxgn.append(i)
        

In [ ]:
bxl_final_hxgn = [ ]
for i in bxl_df_hexagons['geometry']:
    if i.intersects(df_bruxelles['geometry'][0][0])==True:
        bxl_final_hxgn.append(i)
        

In [ ]:
ander_final_hxgn = [ ]
for i in ander_df_hexagons['geometry']:
    if i.intersects(df_anderlecht['geometry'][0][0])==True:
        ander_final_hxgn.append(i)

In [ ]:
ville_final_hxgn = [ ]
for i in ville_df_hexagons['geometry']:
    if i.intersects(df_ville['geometry'][0][0])==True:
        ville_final_hxgn.append(i)

In [ ]:
bel_final_sqr = [ ]
for i in cell_bel['geometry']:
    if i.intersects(df_belgium['geometry'][0])==True:
        bel_final_sqr.append(i)
        

In [ ]:
bxl_final_sqr = [ ]
for i in cell_bxl['geometry']:
    if i.intersects(df_bruxelles['geometry'][0][0])==True:
        bxl_final_sqr.append(i)
        

### transform in geopandas and set CRS

In [ ]:
hxg_bel = gpd.GeoDataFrame(bel_final_hxgn, columns=['geometry'])
hxg_bel = hxg_bel.set_crs(epsg=4326)
hxg_bel.head()

In [ ]:
hxg_bxl = gpd.GeoDataFrame(bxl_final_hxgn, columns=['geometry'])
hxg_bxl = hxg_bxl.set_crs(epsg=4326)
hxg_bxl.head()

In [ ]:
hxg_ander = gpd.GeoDataFrame(ander_final_hxgn, columns=['geometry'])
hxg_ander = hxg_ander.set_crs(epsg=4326)
hxg_ander.head()

In [ ]:
hxg_ville = gpd.GeoDataFrame(ville_final_hxgn, columns=['geometry'])
hxg_ville = hxg_ville.set_crs(epsg=4326)
hxg_ville.head()

In [ ]:
sqr_bel = gpd.GeoDataFrame(bel_final_sqr, columns=['geometry'])
sqr_bel = sqr_bel.set_crs(epsg=4326)
sqr_bel.head()

In [ ]:
sqr_bxl = gpd.GeoDataFrame(bxl_final_sqr, columns=['geometry'])
sqr_bxl = sqr_bxl.set_crs(epsg=4326)
sqr_bxl.head()

### visualiza grid in folium

In [ ]:
m = folium.Map([50.85045, 4.34878], zoom_start=13)
folium.GeoJson(hxg_ville).add_to(m)
folium.GeoJson(df_ville).add_to(m)
folium.GeoJson(hxg_ander).add_to(m)
folium.GeoJson(df_anderlecht).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
m = folium.Map([50.85045, 4.34878], zoom_start=8)

marker_cluster = MarkerCluster().add_to(m)

for coord in range(len(df_geo.iloc[:1000])):
    folium.Marker( location=[df[3][coord], df[2][coord] ]).add_to(marker_cluster)

folium.GeoJson(hxg_bel).add_to(m)
folium.GeoJson(sqr_bel).add_to(m)
folium.GeoJson(hxg_bxl).add_to(m)
folium.GeoJson(sqr_bxl).add_to(m)
folium.GeoJson(df_bruxelles).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
    
    
m

# pydeck visualization

## install and import pydeck

In [ ]:
!pip install pydeck
import pydeck as pdk

In [ ]:
INITIAL_VIEW_STATE = pdk.ViewState(latitude=50.85045, longitude=4.34878, zoom=10, max_zoom=14, pitch=45, bearing=0)
geojson = pdk.Layer(
"GeoJsonLayer",
hxg_bxl,
stroked=False,
filled=True,
extruded=True,
wireframe=True,
get_elevation= 10,
get_line_color=[255, 255, 255],
get_fill_color=[255, 255, 255]
)

r = pdk.Deck(layers=geojson, initial_view_state=INITIAL_VIEW_STATE)
r.to_html("geojson_layer.html", notebook_display=True)

## generate column "random traffic" to plot traffic on map

In [ ]:
hxg_bxl['random_traffic'] = np.random.randint(1,250,size=hxg_bxl.shape[0])

In [ ]:

INITIAL_VIEW_STATE = pdk.ViewState(latitude=50.85045, longitude=4.34878, zoom=10, max_zoom=14, pitch=45, bearing=0)
geojson = pdk.Layer(
"GeoJsonLayer",
hxg_bxl,
stroked=False,
filled=True,
extruded=True,
wireframe=True,
get_elevation = "random_traffic*30",
get_fill_color='[255, (1-random_traffic/250)*255, 0]',
get_line_color='[255, 255, 255]',
)

r = pdk.Deck(layers=geojson, initial_view_state=INITIAL_VIEW_STATE)
r.to_html("geojson_layer.html", notebook_display=True)

In [ ]:
hxg_bel.to_file('Belgium_hexagons.json', driver="GeoJSON")
hxg_bxl.to_file('Bruxelles_hexagons.json', driver="GeoJSON")

hxg_ander.to_file('Anderlecht_hexagons.json', driver="GeoJSON")
hxg_ville.to_file('Ville_hexagons.json', driver="GeoJSON")

sqr_bel.to_file('Belgium_quadrants.json', driver="GeoJSON")
sqr_bxl.to_file('Bruxelles_quadrants.json', driver="GeoJSON")